In [39]:
import numpy as np
import pymongo

In [40]:
# Define the range for each variable
min_iot_devices = 10
max_iot_devices = 1000

min_edge_devices = 5
max_edge_devices = 200

min_fog_devices = 2
max_fog_devices = 50

min_cloud_devices = 1
max_cloud_devices = 20

min_idle_power = 10
max_idle_power = 50

min_max_power = 100
max_max_power = 300

min_max_instructions = 1000000
max_max_instructions = 5000000

min_net_delay_edge = 1
max_net_delay_edge = 10

min_net_delay_fog = 5
max_net_delay_fog = 30

min_net_delay_cloud = 20
max_net_delay_cloud = 100

# Define the range for the number of guest machines per host machine
min_guest_per_host = 1
max_guest_per_host = 5 

# Define the number of configurations (n)
n = 100

In [41]:
client_url = "mongodb://localhost:27017/"
db_name = "simulations"
collection_name = "networks"

In [42]:
# Initialize a dictionary to store the generated configurations
configurations = []

# Connect to MongoDB
client = pymongo.MongoClient(client_url)
db = client[db_name]
collection = db[collection_name] 

# Generate n random configurations
for i in range(n):
    config = {
        '_id': f'config-{i + 1}',  # Unique configuration ID
        'iot_devices': np.random.randint(min_iot_devices, max_iot_devices + 1),
        'edge_devices': np.random.randint(min_edge_devices, max_edge_devices + 1),
        'fog_devices': np.random.randint(min_fog_devices, max_fog_devices + 1),
        'cloud_devices': np.random.randint(min_cloud_devices, max_cloud_devices + 1),
        'host_machines': {},
    }
    
    # Generate configurations for each layer based on the relative number of machines
    total_machines = (
        config['iot_devices'] + config['edge_devices'] +
        config['fog_devices'] + config['cloud_devices']
    )
    
    # Define the number of host machines for each layer
    num_host_edge = int(config['edge_devices'])
    num_host_fog = int(config['fog_devices'])
    num_host_cloud = int(config['cloud_devices'])
    
    # Generate configurations for guest machines within each host machine
    for layer, num_hosts in [('edge', num_host_edge), ('fog', num_host_fog), ('cloud', num_host_cloud)]:
        host_machines = []
        for _ in range(num_hosts):
            num_guest = np.random.randint(min_guest_per_host, max_guest_per_host + 1)
            guest_machines = []
            for _ in range(num_guest):
                # Generate separate values for net_delay_request and net_delay_response based on the layer
                net_delay_request = np.random.uniform(min_net_delay_edge, max_net_delay_edge) if layer == 'edge' else np.random.uniform(min_net_delay_fog, max_net_delay_fog) if layer == 'fog' else np.random.uniform(min_net_delay_cloud, max_net_delay_cloud)
                net_delay_response = np.random.uniform(min_net_delay_edge, max_net_delay_edge) if layer == 'edge' else np.random.uniform(min_net_delay_fog, max_net_delay_fog) if layer == 'fog' else np.random.uniform(min_net_delay_cloud, max_net_delay_cloud)

                guest_config = {
                    'idle_cpu_utilization': np.random.uniform(min_idle_power, max_idle_power),
                    'max_cpu_utilization': np.random.uniform(min_max_power, max_max_power),
                    'max_instructions_per_second': np.random.randint(min_max_instructions, max_max_instructions + 1),
                    'net_delay_request': net_delay_request,
                    'net_delay_response': net_delay_response
                }
                guest_machines.append(guest_config)
            host_machines.append({'guest_machines': guest_machines})
        config['host_machines'][layer] = host_machines

    # Insert the configuration into MongoDB
    collection.insert_one(config)
    configurations.append(config)

# Close the MongoDB connection
client.close()

In [47]:
# Define classes to represent the hierarchy
class GuestMachine:
    def __init__(self, data):
        self.idle_cpu_utilization = data['idle_cpu_utilization']
        self.max_cpu_utilization = data['max_cpu_utilization']
        self.max_instructions_per_second = data['max_instructions_per_second']
        self.net_delay_request = data['net_delay_request']
        self.net_delay_response = data['net_delay_response']

class HostMachine:
    def __init__(self, data):
        self.guest_machines = [GuestMachine(guest_data) for guest_data in data['guest_machines']]

class Layer:
    def __init__(self, data):
        self.host_machines = [HostMachine(host_data) for host_data in data]

class Configuration:
    def __init__(self, data):
        self.iot_devices = data['iot_devices']
        self.edge_devices = data['edge_devices']
        self.fog_devices = data['fog_devices']
        self.cloud_devices = data['cloud_devices']
        self.host_machines = {
            'edge': Layer(data['host_machines']['edge']),
            'fog': Layer(data['host_machines']['fog']),
            'cloud': Layer(data['host_machines']['cloud'])
        }

In [48]:
class NetworkConfigurationManager:
    def __init__(self, client_url, db_name, collection_name):
        self.client = pymongo.MongoClient(client_url)
        self.db = self.client[db_name]
        self.collection = self.db[collection_name]

    def get_configuration_by_id(self, config_id):
        config_data = self.collection.find_one({'_id': config_id})
        if config_data:
            return Configuration(config_data)
        else:
            return None

    def get_all_configurations(self):
        configs = []
        cursor = self.collection.find({})
        for config_data in cursor:
            configs.append(Configuration(config_data))
        return configs

    def close_connection(self):
        self.client.close()

In [51]:
network_config_manager = NetworkConfigurationManager(client_url, db_name, collection_name)

In [52]:
all_configs = network_config_manager.get_all_configurations()
for i, config in enumerate(all_configs, start=1):
    print(f"Configuration {i}:")
    print(f"IoT Devices: {config.iot_devices}")
    print(f"Edge Devices: {config.edge_devices}")
    print(f"Fog Devices: {config.fog_devices}")
    print(f"Cloud Devices: {config.cloud_devices}")

Configuration 1:
IoT Devices: 375
Edge Devices: 12
Fog Devices: 14
Cloud Devices: 19
Configuration 2:
IoT Devices: 793
Edge Devices: 128
Fog Devices: 40
Cloud Devices: 9
Configuration 3:
IoT Devices: 732
Edge Devices: 98
Fog Devices: 7
Cloud Devices: 4
Configuration 4:
IoT Devices: 450
Edge Devices: 62
Fog Devices: 30
Cloud Devices: 16
Configuration 5:
IoT Devices: 624
Edge Devices: 8
Fog Devices: 27
Cloud Devices: 12
Configuration 6:
IoT Devices: 393
Edge Devices: 169
Fog Devices: 47
Cloud Devices: 16
Configuration 7:
IoT Devices: 591
Edge Devices: 112
Fog Devices: 10
Cloud Devices: 1
Configuration 8:
IoT Devices: 331
Edge Devices: 164
Fog Devices: 41
Cloud Devices: 10
Configuration 9:
IoT Devices: 315
Edge Devices: 44
Fog Devices: 26
Cloud Devices: 13
Configuration 10:
IoT Devices: 711
Edge Devices: 13
Fog Devices: 26
Cloud Devices: 5
Configuration 11:
IoT Devices: 125
Edge Devices: 91
Fog Devices: 43
Cloud Devices: 4
Configuration 12:
IoT Devices: 987
Edge Devices: 25
Fog Devices: 4